In [9]:
#Required Libraries

# o

import pandas as pd
import re
import os
import nltk

from nltk.corpus import stopwords
    

1. put original column.
2. add new column. 1. for key word 2. for title
3. drop duplicates function will come at last.
4. 



    

In [10]:

#Importing datasets from local machines

file_list = [r"C:\Users\priya\OneDrive\Desktop\Project\dataset_quora-email_2025-06-10_15-49-46-158.csv",
            r"C:\Users\priya\OneDrive\Desktop\Project\dataset_quora-email_2025-06-10_21-12-50-032.csv",
            r"C:\Users\priya\OneDrive\Desktop\Project\dataset_quora-email_2025-06-10_21-20-42-684.csv"]

dataset_list = []     #creating one list for all datasets

for file in file_list:
    df = pd.read_csv(file)
    dataset_list.append(df)
#print(df)


combine_data = pd.concat(dataset_list, ignore_index =  True)    #merging dataset in one list.

print(combine_data.head())  #shows first five line in the output
print(combine_data.shape)   #shows number of rows and column 
print(combine_data.info)    #shows properties of dataframe
    




           keyword                                                url  \
0  AI in marketing  https://www.quora.com/Can-AI-marketing-tools-b...   
1  AI in marketing  https://www.quora.com/How-can-AI-prompts-impro...   
2  AI in marketing  https://www.quora.com/What-are-emerging-career...   
3  AI in marketing  https://www.quora.com/What-are-the-latest-tren...   
4  AI in marketing  https://www.quora.com/How-can-businesses-stay-...   

                                               title  
0  Can AI marketing tools benefit me, even with l...  
1   How can AI prompts improve marketing strategies?  
2  What are emerging career roles in AI-driven di...  
3  What are the latest trends in performance mark...  
4  How can businesses stay ahead in digital marke...  
(150, 3)
<bound method DataFrame.info of                             keyword  \
0                   AI in marketing   
1                   AI in marketing   
2                   AI in marketing   
3                   AI in marketin

In [11]:

#Converting all data into lowercase
# combine_data = combine_data.astype(str).apply(lambda x: x.str.lower())   #where x tends to every column and astype means converting data into strings 

# print(combine_data.head())


# combine_data['title_lower'] = combine_data['title'].astype(str).apply(lambda x: x.lower())
# print(combine_data[['title', 'title_lower']].head())


In [12]:
#removing special characters and symbols


#df['title'] = df['title'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x) if isinstance(x, str) else x)

#combine_data['title'] = combine_data['title'].apply(lambda x: re.sub(r'[a-z0-9\s]', '', x) if isinstance(x, str) else x)


def clean_data(text):
    if pd.isnull(text):
        return ' '  
    text = str(text)  # convert to string, in case it's not     
    text = text.lower()    
    text = text.encode("ascii", "ignore").decode()
    text=re.sub(r'[^a-z0-9\s]', '',text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
combine_data['title_cleaned']=combine_data['title'].apply(clean_data)

combine_data['keyword_cleaned']=combine_data['keyword'].apply(clean_data)


print(combine_data[['title','title_cleaned']].head())




                                               title  \
0  Can AI marketing tools benefit me, even with l...   
1   How can AI prompts improve marketing strategies?   
2  What are emerging career roles in AI-driven di...   
3  What are the latest trends in performance mark...   
4  How can businesses stay ahead in digital marke...   

                                       title_cleaned  
0  can ai marketing tools benefit me even with li...  
1    how can ai prompts improve marketing strategies  
2  what are emerging career roles in aidriven dig...  
3  what are the latest trends in performance mark...  
4  how can businesses stay ahead in digital marke...  


In [13]:
# Removing extra and unwanted space.

# def remove_extra_space(text):
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text
# def remove_extra_space(text):
#     if pd.isnull(text):
#         return ''  # handle missing values
#     text = text.lower()       
#     text = str(text)  # convert to string, in case it's not
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text
    
# combine_data['space_removed'] = combine_data['title'].apply(remove_extra_space)

# print(combine_data[['title','space_removed']].head())



In [14]:
# #removing Stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


def remove_stopwords(text):
    if pd.isnull(text):
        return ''
    text = text.lower()       
    words = text.split()
    filtered = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered)
    
combine_data['stopwords_removed'] = combine_data['title'].apply(remove_stopwords)
print(combine_data[['title','stopwords_removed']].head())

df.to_csv('clean02_data.csv', index=False)

                                               title  \
0  Can AI marketing tools benefit me, even with l...   
1   How can AI prompts improve marketing strategies?   
2  What are emerging career roles in AI-driven di...   
3  What are the latest trends in performance mark...   
4  How can businesses stay ahead in digital marke...   

                                 stopwords_removed  
0  ai marketing tools benefit me, even limited ...  
1         ai prompts improve marketing strategies?  
2      emerging career roles ai-driven digital ...  
3          latest trends performance marketing ...  
4  businesses stay ahead digital marketing trends?  


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Ensure all entries are valid questions:Ends with ? or starts with a WH-word ("What", "How", "Why"...)
wh_words = ['what', 'how', 'why', 'who', 'when', 'where', 'which', 'whom', 'whose']

def is_valid_question(text):
    if pd.isnull(text):
        return False
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(wh) for wh in wh_words)

combine_data['is_valid_question'] = combine_data['title'].astype(str).apply(is_valid_question)

# Preview the result
print(combine_data[['title', 'is_valid_question']].head())


                                               title  is_valid_question
0  Can AI marketing tools benefit me, even with l...              False
1   How can AI prompts improve marketing strategies?               True
2  What are emerging career roles in AI-driven di...               True
3  What are the latest trends in performance mark...               True
4  How can businesses stay ahead in digital marke...               True


In [16]:
#Removing duplication data from datasets

combine_data = combine_data.drop_duplicates(subset=['title_cleaned', 'keyword_cleaned'], keep='first')
print(combine_data[['title_cleaned', 'keyword_cleaned']].head())

                                       title_cleaned  keyword_cleaned
0  can ai marketing tools benefit me even with li...  ai in marketing
1    how can ai prompts improve marketing strategies  ai in marketing
2  what are emerging career roles in aidriven dig...  ai in marketing
3  what are the latest trends in performance mark...  ai in marketing
4  how can businesses stay ahead in digital marke...  ai in marketing


In [17]:
combine_data.to_csv('clean11_data.csv', index=False)